## Imports

In [ ]:
#Data
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import pickle

#Model
from src.layers import Template

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassAccuracy
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_score, recall_score, accuracy_score, f1_score, auc, precision_recall_curve

## Signal functions

In [ ]:
def random_sin(a: float, f: float, p: float, start: float, end: float, n: int, noise: float = 0.1, irregular: bool = False):
    """Genera una señal sinusoidal aleatoria con ruido.

    Args:
        a (float): amplitud
        f (float): Frecuencia (Hz)
        p (float): Fase
        start (float): Tiempo de inicio
        end (float): Tiempo de termino
        n (int): numero de muestras
        noise (float, optional): Varianza del ruido. Defaults to 0.1.
        irregular (bool, optional): Si es True, se generan muestras irregulares. Defaults to False.
    """
    if irregular:
        t = np.sort(np.random.uniform(start, end, n))
    else:
        t = np.linspace(start, end, n)
    y = a * np.sin(2 * np.pi * f * t + p) + np.random.normal(0, noise, n)
    return t, y, 1/f
    

In [ ]:
def random_triangular(a: float, f: float, p: float, start: float, end: float, n: int, noise: float = 0.05, irregular: bool = False):
    """Genera una señal triangular aleatoria con ruido.

    Args:
        a (float): amplitud
        f (float): Frecuencia (Hz)
        p (float): Fase
        start (float): Tiempo de inicio
        end (float): Tiempo de termino
        n (int): numero de muestras
        noise (float, optional): Varianza del ruido. Defaults to 0.1.
        irregulat(bool, optional): Si es True, se generan muestras irregulares. Defaults to False.
    """
    if irregular:
        t = np.sort(np.random.uniform(start, end, n))
    else:
        t = np.linspace(start, end, n)
    y = 4*a*f*np.abs(np.mod(t-1/(4*f)+p, 1/f)-1/(2*f)) - a + np.random.normal(0, noise, n)
    return t, y, 1/f

In [ ]:
def random_square(a: float, f: float, p: float, start: float, end: float, n: int, noise: float = 0.02, irregular: bool = False):
    """Genera una señal cuadrada aleatoria con ruido.

    Args:
        a (float): amplitud
        f (float): Frecuencia (Hz)
        p (float): Fase
        start (float): Tiempo de inicio
        end (float): Tiempo de termino
        n (int): numero de muestras
        noise (float, optional): Varianza del ruido. Defaults to 0.1.
        irregular (bool, optional): Si es True, se generan muestras irregulares. Defaults to False.
    """
    if irregular:
        t = np.sort(np.random.uniform(start, end, n))
    else:
        t = np.linspace(start, end, n)
    y = a * np.sign(np.sin(2 * np.pi * f * t + p)) + np.random.normal(0, noise, n)
    return t, y, 1/f

In [ ]:
def random_sawtooth(a: float, f: float, p: float, start: float, end: float, n: int, noise: float = 0.02, irregular: bool = False):
    """Genera una señal triangular aleatoria con ruido.

    Args:
        a (float): amplitud
        f (float): Frecuencia (Hz)
        p (float): Fase
        start (float): Tiempo de inicio
        end (float): Tiempo de termino
        n (int): numero de muestras
        noise (float, optional): Varianza del ruido. Defaults to 0.1.
        irregular (bool, optional): Si es True, se generan muestras irregulares. Defaults to False.
    """
    if irregular:
        t = np.sort(np.random.uniform(start, end, n))
    else:
        t = np.linspace(start, end, n)
    y = 2*a*(t*f-np.floor(1/2+t*f)) + np.random.normal(0, noise, n)
    return t, y, 1/f

In [ ]:
def random_gaussian_process(a: float, f: float, p: float, start: float, end: float, n: int, noise: float = 0.02, irregular: bool = False):
    """Genera una señal triangular aleatoria con ruido.

    Args:
        a (float): amplitud
        f (float): Frecuencia (Hz)
        p (float): Fase
        start (float): Tiempo de inicio
        end (float): Tiempo de termino
        n (int): numero de muestras
        noise (float, optional): Varianza del ruido. Defaults to 0.1. (Not used)
        irregular (bool, optional): Si es True, se generan muestras irregulares. Defaults to False.
    """
    if irregular:
        t = np.sort(np.random.uniform(start, end, n))
    else:
        t = np.linspace(start, end, n)
    eps1 = np.random.randn(n)
    eps2 = np.random.randn(n)
    y = a*np.cos(2*np.pi*f*t + p)*eps1 + a*np.sin(2*np.pi*f*t + p)*eps2
    return t, y, 1/f

In [ ]:
def random_nonperiodic(a: float, f: float, p: float, start: float, end: float, n: int, noise: float = 0.02, irregular: bool = False):
    """Genera una señal triangular aleatoria con ruido.

    Args:
        a (float): amplitud
        f (float): Frecuencia (Hz)
        p (float): Fase
        start (float): Tiempo de inicio
        end (float): Tiempo de termino
        n (int): numero de muestras
        noise (float, optional): Varianza del ruido. Defaults to 0.1. (Not used)
        irregular (bool, optional): Si es True, se generan muestras irregulares. Defaults to False.
    """
    if irregular:
        t = np.sort(np.random.uniform(start, end, n))
    else:
        t = np.linspace(start, end, n)
    y = (a/2)*np.sin(2*np.pi*f*t + p) + (a/2)*np.sin(2*np.pi*np.sqrt(2)*f*t + p) + np.random.normal(0, noise, n)
    return t, y, 1/f

### Test signal

In [ ]:
amplitude = 5
frequency = 1
phase = 0
start = 0
end = 10
n = 120
noise = amplitude/10
irregular = True

t, y, p = random_triangular(a=amplitude, f=frequency, p=phase,
                    start=start, end=end, n=n,
                    noise=noise, irregular=irregular)
plt.scatter(t, y)

In [ ]:
t_folded = t % p
t_index = np.argsort(t_folded)
plt.scatter(t_folded[t_index], y[t_index])

## Generación de señales

### Parametros

In [ ]:
# Maximum and minimum number of points per signal
low_signal_length = 70
high_signal_length = 120

# Number of samples per class
n_samples_per_signal = 1500

# Parameters
amplitude_lims = (0, 5.)
frecuency_lims = (0.1, 1.)
phase_lims = (-5., 5.)
end_lims = (7., 10.)
noise_percentage = 0.1 # 10% of the amplitude
irregular = True

#Classes
fun_gen = [random_sin, random_triangular, random_square, random_sawtooth, random_nonperiodic, random_gaussian_process]
classes = ['sin', 'triangular', 'square', 'sawtooth', 'nonperiodic', 'gaussian_process']

### Generacion

In [ ]:
times = []
X = []
periods = []
y = []
for class_, fun in enumerate(fun_gen):
    for _ in range(n_samples_per_signal):
        amplitude = random.uniform(*amplitude_lims)
        t, signal, period = fun(a=amplitude,
                f=random.uniform(*frecuency_lims) if fun != random_gaussian_process else 0.7,
                p=random.uniform(*phase_lims),
                start=0, 
                end=random.uniform(*end_lims),
                n=int(random.uniform(low_signal_length, high_signal_length)),
                noise=noise_percentage*amplitude,
                irregular=irregular)
        times.append(t)
        X.append(signal)
        periods.append(period)
        y.append(class_)

### Guardar/cargar datos

In [ ]:
# with open("data.pkl", "wb") as f:
#     pickle.dump((times, X, periods, y), f)

# if os.path.exists("data.pkl"):
#     with open("data.pkl", "rb") as f:
#         times, X, periods, y = pickle.load(f)

## Modelo

### Dataset

In [ ]:
class SignalDataset(Dataset):
    """Dataset for signals
    
    Attributes:
        X (list): List of signals
        y (list): List of labels
        times (list): List of times
        periods (list): List of periods
        mean (float): Mean of the signals
        std (float): Standard deviation of the signals
        max_time (float): Maximum time of the signals
        fold_signal (bool): If True, the signals are folded

    """

    def __init__(self, X, y, times, periods, mean, std, max_time=np.inf, fold=False):
        self.X = X
        self.y = y
        self.times = times
        self.periods = periods
        self.mean = mean
        self.std = std
        self.max_time = max_time
        self.fold_signal = fold

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        t = self.times[idx]
        p = self.periods[idx]
        x = torch.tensor(self.X[idx], dtype = torch.float32)

        if self.fold_signal:
            t, x = self.fold(t, p, x)

        mask = torch.ones_like(x)
        mask[t > self.max_time] = 0

        return {'data': (x-self.mean)/self.std,
                'time': torch.tensor(self.times[idx], dtype = torch.float32),
                'mask': mask}, torch.tensor(self.y[idx], dtype= torch.long)
    
    def fold(self, t, p, x):
        t_folded = t % p
        t_index = np.argsort(t_folded)
        t_folded = t_folded[t_index]
        x_folded = x[t_index]
        
        return t_folded, x_folded


In [ ]:
def collate_fn(batch):
    (xx, yy) = zip(*batch)
    x_data = [x['data'] for x in xx]
    x_time = [x['time'] for x in xx]
    x_mask = [x['mask'] for x in xx]

    x_data_pad = pad_sequence(x_data, batch_first=True, padding_value=0)
    x_time_pad = pad_sequence(x_time, batch_first=True, padding_value=0)
    x_mask_pad = pad_sequence(x_mask, batch_first=True, padding_value=0)

    return {'data': x_data_pad.unsqueeze(2), 'time': x_time_pad.unsqueeze(2), 'mask': x_mask_pad.unsqueeze(2)}, torch.stack(yy)

In [ ]:
def create_datasets(X: list, y: list, times: list, periods: list, test_size: float,
                    max_time: float = np.inf, fold: bool = False):
    """Create train, validation and test datasets.

    Args:
        X (list): Signals
        y (list): Targets
        times (list): Time of each signal
        periods (list): Period of each signal
        test_size (float): Percentage of val+test samples
        max_time (float, optional): Maximum time for the signals . Defaults to np.inf.
        fold (bool, optional): If fold the signal. Defaults to False.

    Returns:
        tuple: Train, validation and test datasets
    """
    #Create partitions
    X_train, X_val_test, y_train, y_val_test, \
    times_train, times_val_test, periods_train, periods_val_test = train_test_split(X, y, times,
                                                                                    periods, test_size=test_size,
                                                                                    random_state=42, stratify=y)

    X_val, X_test, y_val, y_test, \
    times_val, times_test, periods_val, periods_test = train_test_split(X_val_test, y_val_test, times_val_test,
                                                                        periods_val_test, test_size=0.5,
                                                                        random_state=42, stratify=y_val_test)
    
    #Stardardization parameters
    #Calculate train mean
    count = 0
    suma = 0
    for x in X_train:
        count += x.shape[0]
        suma += np.sum(x)
    mean_train = suma/count
    #Calculate train std
    count = 0
    suma = 0
    for x in X_train:
        count += x.shape[0]
        suma += np.sum((x-mean_train)**2)
    std_train = np.sqrt(suma/count)

    #Create datasets
    train_dataset = SignalDataset(X_train, y_train, times_train, periods_train, mean_train, std_train, max_time, fold)
    val_dataset = SignalDataset(X_test, y_test, times_test, periods_val, mean_train,std_train, max_time, fold)
    test_dataset = SignalDataset(X_test, y_test, times_test, periods_test, mean_train,std_train, max_time, fold)

    return train_dataset, val_dataset, test_dataset

### Lightning model

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, **kwargs):
        super().__init__()

        self.save_hyperparameters()

        self.layer = Template(**kwargs)
                                     
        self.lr = kwargs['lr']

        self.train_acc = MulticlassAccuracy(kwargs['num_classes'], average='micro')
        self.val_acc = MulticlassAccuracy(kwargs['num_classes'], average='micro')

    def forward(self, x):
        return self.layer(**x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        batch_acc = self.train_acc(logits, y)
        self.log('train_acc', batch_acc, prog_bar=True, on_step=True, on_epoch=True)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        batch_acc = self.val_acc(logits, y)
        self.log('val_acc', batch_acc, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

### Metricas

In [ ]:
def predict(dataloader, model):
    """Predicts the labels of a dataset.

    Args:
        dataloader (DataLoader): Dataset to predict
        model (Model): Model to use

    Returns:
        tuple: Predicted labels and target labels
    """
    model.eval()
    ys_pred = []
    ys_target = []
    with torch.no_grad():
        for x, y in dataloader:
            logits = model(x)
            ys_pred.append(logits.softmax(dim=-1).cpu().numpy())
            ys_target.append(y.cpu().numpy())
    return np.vstack(ys_pred), np.hstack(ys_target)

In [ ]:
def calculate_metrics(target, pred_scores):
    """Calculate metrics for a classification problem.

    Args:
        target (np.array): Target labels
        pred (np.array): Predicted scores

    Returns:
        tuple: Accuracy, precision, recall, f1, confusion matrix, roc auc and pr auc, roc curves, pr curves
    """
    pred = pred_scores.argmax(axis=1)   
    acc = accuracy_score(target, pred)
    precision = precision_score(target, pred, average='macro')
    recall = recall_score(target, pred, average='macro')
    f1 = f1_score(target, pred, average='macro')
    cm = confusion_matrix(target, pred, normalize='true')

    roc_curves = dict()
    roc_aucs = []
    for i in range(pred_scores.shape[1]):
        fpr, tpr, _ = roc_curve(label_binarize(target, classes=np.arange(pred_scores.shape[1]))[:, i], pred_scores[:, i], pos_label=1)
        roc_curves[i] = (fpr, tpr)
        roc_aucs.append(auc(fpr, tpr))

    roc_auc = np.mean(roc_aucs)
    
    pr_curves = dict()
    pr_aucs = []
    for i in range(pred_scores.shape[1]):
        prec, rec, _ = precision_recall_curve(label_binarize(target, classes=np.arange(pred_scores.shape[1]))[:, i], pred_scores[:, i], pos_label=1)
        pr_curves[i] = (prec, rec)
        pr_aucs.append(auc(rec, prec))

    pr_auc = np.mean(pr_aucs)

    return acc, precision, recall, f1, cm, roc_auc, pr_auc, roc_curves, pr_curves

In [ ]:
def plot_confusion_matrix(cms: list, classes: list, save_path: str=None):
    """Plot confusion matrix.

    Args:
        cms (list): List of confusion matrices
        classes (list): List of classes
        save_path (str, optional): Path to save the figure. Defaults to None.
    """

    plt.figure(figsize=(6, 5))
    if len(cms)==1:
        cm = cms[0]
    else:
        cm = np.mean(cms, axis=0)
        cm_std = np.std(cms, axis=0)

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            texto = '{0:.2f}'.format(cm[i, j]) if len(cms)==1 else '{0:.2f}'.format(cm[i, j]) + '\n$\pm$' + '{0:.2f}'.format(cm_std[i, j])
            plt.text(j, i, texto,
                     horizontalalignment="center",
                     verticalalignment="center", fontsize=11,
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion matrix')

    if save_path:
        plt.savefig(save_path, dpi=100)
    plt.show()

In [ ]:
def plot_roc_curves(roc_curves: dict, classes: list, save_path: str=None):
    """Plot roc curves.

    Args:
        roc_curves (dict): Dictionary of roc curves
        classes (list): List of classes
        save_path (str, optional): Path to save the figure. Defaults to None.
    """
    plt.figure(figsize=(6, 5))
    for i in range(len(classes)):
        fpr, tpr = roc_curves[i]
        plt.plot(fpr, tpr, label=f'{classes[i]}')
    #0.5 chance
    plt.plot([0, 1], [0, 1], 'k--')
    plt.legend()
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curves')
    if save_path:
        plt.savefig(save_path, dpi=100)
    plt.show()

In [ ]:
def plot_pr_curves(pr_curves: dict, classes: list, save_path: str=None):
    """Plot pr curves.

    Args:
        pr_curves (dict): Dictionary of pr curves
        classes (list): List of classes
        save_path (str, optional): Path to save the figure. Defaults to None.
    """

    plt.figure(figsize=(6, 5))
    for i in range(len(classes)):
        prec, rec = pr_curves[i]
        plt.plot(rec, prec, label=f'{classes[i]}')
    plt.legend()
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PR curves')
    if save_path:
        plt.savefig(save_path, dpi=100)
    plt.show()

## Experimentos

### Parametros

In [ ]:
# Dataset
fold_curves = False
max_time = np.inf
test_size = 0.4

# Dataloader
n_workers = 0
batch_size = 200

# Model
args = {"num_bands": 1,
        "embedding_size": 512,
        "embedding_size_sub": 2048,
        "num_heads": 4,
        "num_encoders": 1,
        "Tmax": 15,
        "num_harmonics": 16,
        "num_classes": len(classes),
        'lr': 1e-4}

# Experiment
accelerator = 'gpu' #gpu or cpu
experiment_name = 'testAllCurve'
epochs = 1000
patience = 50
n_execs = 5

### Configurar experimento

In [ ]:
train_dataset, val_dataset, test_dataset = create_datasets(X, y, times, periods, test_size=test_size, max_time=max_time, fold=fold_curves)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=n_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=n_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=n_workers)

### Ejecutar experimento

In [ ]:
ckpt_paths = []
for i in range(n_execs):
    #Callbacks
    early_stop_callback = pl.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=patience, verbose=True, mode='min')
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min')

    #Trainer
    trainer = pl.Trainer(default_root_dir=experiment_name,
                        max_epochs=epochs,
                        accelerator=accelerator,
                        callbacks=[checkpoint_callback,early_stop_callback],
                        log_every_n_steps=1,
                        logger=[pl.loggers.CSVLogger(save_dir=experiment_name)])
    
    #Fit model
    model = Model(**args)
    trainer.fit(model, train_loader, val_loader)
    ckpt_paths.append(checkpoint_callback.best_model_path)

### Evaluar experimento

In [ ]:
# If the evaluation is done in a different moment, put here the path to the checkpoints
# ckpt_paths = []

In [ ]:
loader = test_loader

accs, precisions, recalls, f1s, cms, roc_aucs, pr_aucs, roc_curves, pr_curves = [], [], [], [], [], [], [], [], []
for ckpt in ckpt_paths:
    model = Model.load_from_checkpoint(ckpt)
    pred_scores, target = predict(loader, model)
    acc, precision, recall, f1, cm, roc_auc, pr_auc, roc_curve_exp, pr_curve_exp = calculate_metrics(target, pred_scores)
    accs.append(acc)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    cms.append(cm)
    roc_aucs.append(roc_auc)
    pr_aucs.append(pr_auc)
    roc_curves.append(roc_curve_exp)
    pr_curves.append(pr_curve_exp)

print(f'Accuracy: {np.mean(accs):.3f} \u00B1 {np.std(accs):.3f}')
print(f'Precision: {np.mean(precisions):.3f} \u00B1 {np.std(precisions):.3f}')
print(f'Recall: {np.mean(recalls):.3f} \u00B1 {np.std(recalls):.3f}')
print(f'F1: {np.mean(f1s):.3f} \u00B1 {np.std(f1s):.3f}')
plot_confusion_matrix(cms, classes, save_path=f'{experiment_name}/cm.png')

In [ ]:
f1_sorted = np.argsort(f1s)
median_f1 = f1_sorted[len(f1_sorted)//2]
best_f1 = f1_sorted[-1]

print('Median')
plot_roc_curves(roc_curves[median_f1], classes, save_path=f'{experiment_name}/roc_median.png')
plot_pr_curves(pr_curves[median_f1], classes, save_path=f'{experiment_name}/pr_median.png')

print('Best')
plot_roc_curves(roc_curves[best_f1], classes, save_path=f'{experiment_name}/roc_best.png')
plot_pr_curves(pr_curves[best_f1], classes, save_path=f'{experiment_name}/pr_best.png')

### Curva de aprendizaje

In [ ]:
n_exp = 0
file = f"{experiment_name}/lightning_logs/version_{n_exp}/metrics.csv"

df = pd.read_csv(file)
val_loss = df['val_loss'].dropna()
train_loss = df['train_loss_epoch'].dropna()
epochs = np.arange(len(val_loss))

plt.figure(figsize=(8, 4))
plt.plot(epochs, val_loss, label='val_loss')
plt.plot(epochs, train_loss, label='train_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training curve')
plt.grid()
# plt.savefig(f'{experiment_name}/training_curve.png', dpi=100)
plt.show()